In [ ]:
!pip install kagglehub
!pip install json
!pip install openai

In [ ]:
instruction_for_dataset = "extract the rdf from this message in N-tripple format"
instruction_for_llm = "extract the detailed rdf from this message in N-triple format. Use detailed predicates. Don't copy full text from body. For example, <person:george_richards> <schema:jobTitle> \"President\", <msg:25> <prop:earlyLandCost> \"4000 USD per month\"^^<unit:USDollarPerMonth>. Optimize response for saving tokens"
instruction_for_llm_rus = "Сгенерируй сообщение, похожее на это, чтобы оно было на русском языке"
instruction_for_llm_rus_rdf = "extract the detailed rdf from this message in N-triple format. Use detailed predicates. Don't copy full text from body. For example, <person:george_richards> <schema:jobTitle> \"President\", <msg:25> <prop:earlyLandCost> \"4000 долларов в месяц\"^^<unit:USDollarPerMonth>. Optimize response for saving tokens"

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import json


used_id_file = 'message_ids.txt'
dataset_file = "english_dataset.jsonl"


def check_id(message, used_id):
    lines = message.split('\n')
    for line in lines:
        if ':' in line:
            header_name = line.split(':', 1)[0].strip()
            if header_name == "Message-ID":
                message_id = line.split(':', 1)[1].strip()
                if message_id in used_id:
                    return True
                else:
                    return False



def filter_email_headers(message, used_id_file):
    """Фильтрует заголовки email и обрезает пересланную часть"""
    lines = message.split('\n')
    allowed_headers = {'Date', 'From', 'To'}
    filtered_lines = []
    in_headers = True
    is_original = False

    for line in lines:
        # Проверяем, не началась ли пересланная часть
        if "Forwarded by" in line:
            break

        #поиск нужных заголовков и добавление непустых строк
        if in_headers:
            if ':' in line:
                header_name = line.split(':', 1)[0].strip()
                if header_name in allowed_headers:
                    filtered_lines.append(line)
                elif header_name == "Message-ID":
                    with open(used_id_file, 'a') as file:
                        file.write(str(line.split(':', 1)[1].strip()) + "\n")
            elif line.strip() == "":
                 in_headers = False
        elif line.strip() != "":
                filtered_lines.append(line)
                is_original = True


    if is_original:
        return "\n".join(filtered_lines)
    else:
         return "---Forwarded message---"


def add_data(instruction, input, output, dataset_file):
    new_item = {
        "instruction": instruction,
        "input": input,
        "output": output
    }
    with open(dataset_file, "a", encoding="utf-8") as file:
        file.write(json.dumps(new_item, ensure_ascii=False) + "\n")

def format_string_for_writing(response):
    lines = response.split('\n')
    filtered_lines = []
    for line in lines:
        if line.strip != "":
            filtered_lines.append(line)
    return "    ".join(filtered_lines)


# Set the path to the file you'd like to load
file_path = "emails.csv"

# Load the dataset with iterator to read one message at a time
df_iter = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "wcukierski/enron-email-dataset",
    file_path,
    pandas_kwargs={
        "usecols": ["message"],
        "chunksize": 1  # Читаем по одному сообщению за раз
    }
)

In [ ]:
from openai import OpenAI
import json

client = OpenAI(
    api_key="sk-or-vv-8dc7fde61c37a92124dcf1a9f236066014c49f7ba4231cafa064e40792dfed0e",
    base_url="https://api.vsegpt.ru/v1"
)

In [ ]:
# создание датасета на английском языке
with open('message_ids.txt', 'r') as file:
    # Читаем все строки файла и удаляем символы переноса строки
    used_id = [line.strip() for line in file.readlines()]

counter = 0
for i, chunk in enumerate(df_iter, 1):
    if i <= 2000:
      continue
    input = chunk["message"].iloc[0]
    #print(input)
    if check_id(input, used_id):
        continue
    input = filter_email_headers(input, used_id_file)
    if input != "---Forwarded message---":
        input = f"\n--- Message {i} ---" + "\n" + input
        prompt = instruction_for_llm + ":" + "\n" + input
        messages = []
        #messages.append({"role": "system", "content": system_text})
        messages.append({"role": "user", "content": prompt})
        response_big = client.chat.completions.create(
            #model="deepseek/deepseek-chat-0324-alt",
            model="openai/gpt-4.1-nano",
            messages=messages,
            temperature=0.7,
            n=1,
            max_tokens=3000
        )
        #print("Response BIG:",response_big)
        response = response_big.choices[0].message.content
        print(input)
        print(response)
        response = format_string_for_writing(response)
        input = format_string_for_writing(input)
        add_data(instruction_for_dataset, input, response, dataset_file)
        print("==ok==")

    counter += 1
    if counter == 500:
        break

    # Если нужно ограничить количество сообщений
    if i >= 4000:
        break

In [ ]:
# перевод датасета на русский язык
with open('message_ids_rus.txt', 'r') as file:
    # Читаем все строки файла и удаляем символы переноса строки
    used_id = [line.strip() for line in file.readlines()]

counter = 0
for i, chunk in enumerate(df_iter, 1):
    if i <= 1000:
        continue
    # Извлекаем первое (и единственное) сообщение из чанка
    input = chunk["message"].iloc[0]
    #print(input)
    if check_id(input, used_id):
        continue
    input = filter_email_headers(input, used_id_file)
    if input != "---Forwarded message---":
        input = f"\n--- Message {i} ---" + "\n" + input
        prompt = instruction_for_llm_rus + ":" + "\n" + input
        messages = []
        #messages.append({"role": "system", "content": system_text})
        messages.append({"role": "user", "content": prompt})
        response_big = client.chat.completions.create(
            #model="deepseek/deepseek-chat-0324-alt",
            model="openai/gpt-4.1-nano",
            messages=messages,
            temperature=0.7,
            n=1,
            max_tokens=3000
        )
        #print("Response BIG:",response_big)
        response = response_big.choices[0].message.content
        print(response)
        response = format_string_for_writing(response)
        input = format_string_for_writing(input)
        add_data(instruction_for_dataset, input, response, dataset_file)
        print("==ok==")


    counter += 1
    if counter == 300:
      break

    # Если нужно ограничить количество сообщений
    if i >= 2000:
        break

In [ ]:
# создание датасета на русском языке
def extract_attribute_from_jsonl(file_path, attribute_name):
    values = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            if attribute_name in data:
                values.append(data[attribute_name])
    return values



file_path = "/content/new_russian_dataset.jsonl"
attribute_name = "output"
extracted_values = extract_attribute_from_jsonl(file_path, attribute_name)

for i in range(len(extracted_values)):
    # Извлекаем первое (и единственное) сообщение из чанка
    input = extracted_values[i]
    #print(input)
    prompt = instruction_for_llm_rus_rdf + ":" + "\n" + input
    messages = []
    #messages.append({"role": "system", "content": system_text})
    messages.append({"role": "user", "content": prompt})
    response_big = client.chat.completions.create(
        #model="deepseek/deepseek-chat-0324-alt",
        model="openai/gpt-4.1-nano",
        messages=messages,
        temperature=0.7,
        n=1,
        max_tokens=3000
    )
    #print("Response BIG:",response_big)
    response = response_big.choices[0].message.content
    print(prompt)
    print("=================")
    print(response)
    response = format_string_for_writing(response)
    input = format_string_for_writing(input)
    add_data(instruction_for_dataset, input, response, dataset_file)
    print("==ok==")



    # Если нужно ограничить количество сообщений
    #if i >= 670:
       # break